In [1]:
import os
from sqlalchemy import create_engine
import pybaseball as pyb
import pybaseball.cache # Ensure caching is imported
import pandas as pd
from dotenv import load_dotenv
import time
from datetime import date, timedelta
from sqlalchemy.engine import Engine
from sqlalchemy import text
import datetime
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException
from rapidfuzz import process
import re
import numpy as np
from io import StringIO
import pylahman
import statsapi

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:

# Load environment variables from .env file
load_dotenv()

# Build the PostgreSQL connection string
DB_URL = f"postgresql://{os.environ['DB_USER']}:{os.environ['DB_PASS']}@{os.environ['DB_HOST']}:5432/{os.environ['DB_NAME']}"

# Create the engine object for connecting
engine = create_engine(DB_URL)

print("Database connection established.")

Database connection established.


Test - Pitcher archetype

In [ ]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler


def run_scouting_model(df):
    """
    Synthesizes pitching identity (GMM) with performance outcomes (Whiff/Barrel),
    Perceived Power (Extension), and Vertical Separation.
    """
    
    # 1. Identity Features (Physical & Tactical only)
    identity_features = [
        'ffour_usage', 'sinker_usage', 'bb_usage', 'offspeed_usage',
        'ffour_vaa_pct', 'sinker_vaa_pct', 'bb_vaa_pct', 'offspeed_vaa_pct',
        'velo_gap_pct', 'command_pct', 'paint_pct'
    ]
    
    # 2. Effectiveness Scores
    df['lethality_score'] = (
        (df['whiff_pct'] * 0.75) + 
        (df['suppression_pct'] * 0.20) + 
        (df['velo_pct'] * 0.05)
    ).round(1)

    # 3. Clustering (Archetype Definition)
    scaler = StandardScaler()
    scaled_identity = scaler.fit_transform(df[identity_features].fillna(0))
    
    gmm = GaussianMixture(n_components=7, random_state=42)
    df['style_cluster'] = gmm.fit_predict(scaled_identity)

    # 4. Outlier Detection (Unicorns)
    iso = IsolationForest(contamination=0.04, random_state=42)
    df['is_unicorn'] = iso.fit_predict(scaled_identity)
    
    def profile_pitcher_clusters(df):
        # Defining the tactical metrics we want to see
        # profile_metrics = [
        #     'style_cluster', 'fb_velo', 'ffour_usage', 'sinker_usage', 
        #     'offspeed_usage', 'whiff_pct', 'paint_pct', 'vaa_plus_pct'
        # ]
        
        profile_metrics = [
        'style_cluster', 
        # DNA
        'ffour_usage', 'sinker_usage', 'ffour_vaa_pct', 'velo_gap_pct', 'paint_pct',
        # RESULTS
        'fb_velo', 'whiff_pct', 'suppression_pct']
        
        # Calculate the mean for each cluster
        profile = df[profile_metrics].groupby('style_cluster').mean().round(2)
        
        # Sort by whiff_pct to see the "Dominance Hierarchy"
        #return profile.sort_values(by='whiff_pct', ascending=False)
        return df[profile_metrics].groupby('style_cluster').mean().round(2)
    
    # Usage:
    pitcher_profile_table = profile_pitcher_clusters(df)
    print(pitcher_profile_table)
    
    def calculate_pitcher_grade(row):
        # 1. STUFF+ (Physicality)
        # This is the "Weapon" - 60% of the overall grade
        stuff_plus = row['stuff_plus_pct']
        
        # 2. LOCATION+ (Surgicality)
        # This is the "Aim" - 40% of the overall grade
        location_plus = row['location_plus_pct']
        
        # 3. PITCHING+ (The Master Score)
        # Note: We weigh Stuff higher because it's harder to find/teach
        base_score = (stuff_plus * 0.60) + (location_plus * 0.40)
        
        # 4. VOLUME/STARTER ADJUSTMENTS
        if row['is_starter'] == 1:
            base_score += 5  # The "Skubal Boost"
        
        if row['total_appearances'] < 5:
            base_score -= 10 # The "Sample Size Penalty"

        # 5. FINAL LETTER GRADE
        if base_score >= 85: 
            grade = 'A+'
        elif base_score >= 75: 
            grade = 'A'
        elif base_score >= 60: 
            grade = 'B'
        elif base_score >= 45: 
            grade = 'C'
        else: 
            grade = 'F'
            
        return grade, stuff_plus, location_plus

    #df['overall_grade'] = df.apply(calculate_pitcher_grade, axis=1)
    df[['overall_grade', 'stuff_plus_final', 'location_plus_final']] = df.apply(
        lambda x: pd.Series(calculate_pitcher_grade(x)), axis=1
    )


    def generate_scouting_report(row):
        tags = []
        # Start the summary with the Grade and Handedness
        summary_header = f"[{row['overall_grade']} GRADE] ({row['hand']})"
        
        # 1. CORE IDENTITY TAGS (Physicality)
        s_plus = row['stuff_plus_pct']
        l_plus = row['location_plus_pct']
        
        if s_plus >= 90: tags.append("💣 PURE FILTH")
        elif s_plus <= 20: tags.append("📉 LACKS BITE")

        if l_plus >= 90: tags.append("🎯 SURGEON")
        elif l_plus <= 20: tags.append("🏹 WILD THING")

        # 2. MATCHUP TACTICS (New Logic)
        # We use the columns we just built in SQL
        profile = row['attack_profile']
        role = row['matchup_role']
        platoon = row['platoon_identity']
        
        # Build the Narrative Summary
        analysis = f"Identified as a {role}. "
        
        if "NORTH-SOUTH" in profile:
            analysis += "Wins vertically with high-carry fastballs; elite matchup against low-ball hitters. "
        elif "EAST-WEST" in profile:
            analysis += "Heavy horizontal movement profile; ideal for inducing double plays. "
        
        if platoon == "MATCHUP PROOF":
            tags.append("🛡️ PLATOON NEUTRAL")
            analysis += "Maintains effectiveness regardless of batter handedness. "
        elif platoon == "PLATOON SENSITIVE":
            tags.append("⚠️ SPLIT RISK")
            analysis += "Performance drops significantly against opposite-handed hitters. "

        # 3. SPECIAL TRAITS
        if row['tunnel_pct'] >= 90: tags.append("🧬 TUNNELER")
        if row['is_unicorn'] == -1: tags.append("🦄 UNICORN")
        if row['breakout_potential'] != 'OPTIMIZED':
            tags.append("🚀 BREAKOUT")
            analysis += f"Tactical Alert: {row['breakout_potential']}. "

        # Create the final string
        tag_str = " | ".join(list(set(tags)))
        final_summary = f"{summary_header} {tag_str} — {analysis.strip()}"
        
        return tag_str, final_summary

    # Apply to your DataFrame
    results = df.apply(generate_scouting_report, axis=1)
    df['archetype_tags'], df['scouting_summary'] = zip(*results)

    # Apply and split into two columns
    results = df.apply(generate_scouting_report, axis=1)
    df['archetype_tags'], df['scouting_summary'] = zip(*results)
    
    return df


def update_dim_pitcher_archetypes(engine):
    """
    SQL to extract the necessary metrics for the Python model.
    """
    query = text("""
    WITH attack_zone_stats AS (
    SELECT 
        p.*,
        -- Define Command/Paint Zones
        CASE 
            WHEN ABS(p.plate_x) <= 0.67 AND p.plate_z BETWEEN (p.sz_bot + 0.33) AND (p.sz_top - 0.33) THEN 'heart'
            WHEN ABS(p.plate_x) <= 1.1 AND p.plate_z BETWEEN (p.sz_bot - 0.33) AND (p.sz_top + 0.33) THEN 'shadow'
            WHEN ABS(p.plate_x) <= 1.5 AND p.plate_z BETWEEN (p.sz_bot - 0.75) AND (p.sz_top + 0.75) THEN 'chase'
            ELSE 'waste'
        END as attack_zone,
        CASE WHEN p.description IN ('swinging_strike', 'swinging_strike_blocked', 'missed_bunt') THEN 1 ELSE 0 END as is_whiff,
        CASE WHEN p.description IN ('swinging_strike', 'swinging_strike_blocked', 'missed_bunt', 'foul', 'foul_tip', 'hit_into_play') THEN 1 ELSE 0 END as is_swing
    FROM fact_statcast_pitches p
    ),
    vaa_base_calc AS (
        SELECT 
            az.*,
            CASE WHEN az.pitch_type IN ('FA', 'FF', 'FC') THEN 
                -ATAN((az.vz0 + (az.az * ((-az.vy0 - SQRT(az.vy0^2 - (2 * az.ay * (50 - (17/12))))) / az.ay))) / 
                (-SQRT(az.vy0^2 - (2 * az.ay * (50 - (17/12)))))) * (180/3.14159) 
            END as individual_ff_vaa
        FROM attack_zone_stats az
    ),
    aggregated_stats AS (
        SELECT 
            p.pitcher,
            p.p_throws,
            COUNT(*) as total_pitches,        
            AVG(p.release_extension) as avg_extension,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('FA', 'FF', 'FT', 'FC', 'SI') THEN p.release_speed + ((p.release_extension - 6.2) * 2) END)::numeric, 1), 0) as perceived_fb_velo,
            (AVG(CASE WHEN p.pitch_type IN ('FA', 'FF') THEN p.pfx_z * 12 END) - 
            AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'SI') THEN p.pfx_z * 12 END)) as v_break_gap_raw,
            
            ROUND(100.0 * SUM(p.is_whiff) / NULLIF(SUM(p.is_swing), 0), 2) as whiff_rate_raw,
            ROUND(100.0 * SUM(CASE WHEN p.launch_speed_angle = 6 THEN 1 ELSE 0 END) / 
                NULLIF(SUM(CASE WHEN p.type = 'X' THEN 1 ELSE 0 END), 0), 2) as barrel_rate_raw,        
            
            ROUND(100.0 * SUM(CASE WHEN p.attack_zone = 'shadow' THEN 1 ELSE 0 END) / COUNT(*), 1) as paint_raw,
            ROUND(100.0 * SUM(CASE WHEN p.attack_zone IN ('shadow', 'chase') THEN 1 ELSE 0 END) / COUNT(*), 1) as command_raw,        
            
            AVG(CASE WHEN p.stand = 'L' THEN p.estimated_woba_using_speedangle END) as xwoba_vs_lhb,
            AVG(CASE WHEN p.stand = 'R' THEN p.estimated_woba_using_speedangle END) as xwoba_vs_rhb,           
            
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('FA', 'FF', 'FT', 'FC', 'SI') THEN p.release_speed END)::numeric, 1), 0) as fb_velo,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'ST', 'SL', 'KC', 'GY', 'SV', 'CS', 'KN', 'EP') THEN p.release_speed END)::numeric, 1), 0) as offspeed_velo,                                                                                                                                                                    
            
            ROUND(100.0 * SUM(CASE WHEN p.pitch_type IN ('FA', 'FF', 'FC') THEN 1 ELSE 0 END) / COUNT(*), 1) as ffour_usage,
            ROUND(100.0 * SUM(CASE WHEN p.pitch_type IN ('SI', 'FT') THEN 1 ELSE 0 END) / COUNT(*), 1) as sinker_usage,
            ROUND(100.0 * SUM(CASE WHEN p.pitch_type IN ('CU', 'SL', 'KC', 'ST', 'SV', 'CS', 'KN') THEN 1 ELSE 0 END) / COUNT(*), 1) as bb_usage,
            ROUND(100.0 * SUM(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'ST', 'SL', 'KC', 'GY', 'SV', 'CS', 'KN', 'EP') THEN 1 ELSE 0 END) / COUNT(*), 1) as offspeed_usage,        
            
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('FA', 'FF', 'FT', 'FC', 'SI') THEN p.release_speed END)::numeric - 
                        AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'ST', 'SL', 'KC', 'GY', 'SV', 'CS', 'KN', 'EP') THEN p.release_speed END)::numeric, 1), 0) as velo_gap,        
            COALESCE(ROUND(AVG(p.individual_ff_vaa)::numeric, 2), 0) as ffour_vaa,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('SI', 'FT') THEN -ATAN((p.vz0 + (p.az * ((-p.vy0 - SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12))))) / p.ay))) / (-SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12)))))) * (180/3.14159) END)::numeric, 2), 0) as sinker_vaa,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('CU', 'SL', 'KC', 'ST', 'SV', 'CS', 'KN') THEN -ATAN((p.vz0 + (p.az * ((-p.vy0 - SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12))))) / p.ay))) / (-SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12)))))) * (180/3.14159) END)::numeric, 2), 0) as bb_vaa,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'EP') THEN -ATAN((p.vz0 + (p.az * ((-p.vy0 - SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12))))) / p.ay))) / (-SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12)))))) * (180/3.14159) END)::numeric, 2), 0) as offspeed_vaa,

            COUNT(DISTINCT game_pk) as total_appearances,
            (COUNT(*) / COUNT(DISTINCT game_pk)) as avg_pitches_per_app,
            CASE WHEN (COUNT(*) / COUNT(DISTINCT game_pk)) >= 40 AND COUNT(DISTINCT game_pk) >= 3 THEN 1 ELSE 0 END as is_starter,
            
            STDDEV(p.release_pos_x) as release_x_std,
            STDDEV(p.release_pos_z) as release_z_std,
            (STDDEV(p.release_pos_x) + STDDEV(p.release_pos_z)) as tunnel_raw,
            AVG(p.individual_ff_vaa - ((-0.68 * p.plate_z) - 3.8)) as vaa_above_expected_raw,       
            -- RAW STUFF+ (Process)
            ( (AVG(p.release_speed) * 0.4) + (AVG(p.release_extension) * 0.2) + (AVG(ABS(p.pfx_x)) * 12 * 0.2) + (AVG(p.pfx_z) * 12 * 0.2) ) as stuff_raw,
            -- RAW LOCATION+ (Process)
            ( (SUM(CASE WHEN p.attack_zone = 'shadow' THEN 1 ELSE 0 END)::float / COUNT(*)) * 0.6 + (SUM(CASE WHEN p.attack_zone = 'heart' THEN 0 ELSE 1 END)::float / COUNT(*)) * 0.4 ) as location_raw

        FROM vaa_base_calc p
        GROUP BY p.pitcher, p.p_throws
        HAVING COUNT(*) > 100 AND AVG(p.release_speed) > 84
    ),
    ranked_stats AS (
        SELECT 
            ast.*,
            ROUND((PERCENT_RANK() OVER (ORDER BY fb_velo))::numeric, 2) * 100 as velo_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (offspeed_usage > 0) ORDER BY offspeed_velo))::numeric, 2) * 100, 0) as offspeed_velo_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (offspeed_usage > 0) ORDER BY velo_gap))::numeric, 2) * 100, 0) as velo_gap_pct,                   
            ROUND((PERCENT_RANK() OVER (ORDER BY whiff_rate_raw))::numeric, 2) * 100 as whiff_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY barrel_rate_raw DESC))::numeric, 2) * 100 as suppression_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY command_raw))::numeric, 2) * 100 as command_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY paint_raw))::numeric, 2) * 100 as paint_pct,            
            ROUND((PERCENT_RANK() OVER (ORDER BY perceived_fb_velo))::numeric, 2) * 100 as perceived_velo_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (offspeed_usage > 0 OR sinker_usage > 0) ORDER BY v_break_gap_raw))::numeric, 2) * 100, 0) as movement_gap_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY avg_extension))::numeric, 2) * 100 as extension_pct,           
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (ffour_usage > 0) ORDER BY ffour_vaa))::numeric, 2) * 100, 0) as ffour_vaa_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (sinker_usage > 0) ORDER BY sinker_vaa DESC))::numeric, 2) * 100, 0) as sinker_vaa_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (bb_usage > 0) ORDER BY bb_vaa DESC))::numeric, 2) * 100, 0) as bb_vaa_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (offspeed_usage > 0) ORDER BY offspeed_vaa DESC))::numeric, 2) * 100, 0) as offspeed_vaa_pct,
            ROUND((100 - (ABS(COALESCE(xwoba_vs_lhb, 0.320) - COALESCE(xwoba_vs_rhb, 0.320)) * 100))::numeric, 2) as neutrality_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY tunnel_raw DESC))::numeric, 2) * 100 as tunnel_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY vaa_above_expected_raw))::numeric, 2) * 100 as vaa_plus_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY stuff_raw))::numeric, 2) * 100 as stuff_plus_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY location_raw))::numeric, 2) * 100 as location_plus_pct
        FROM aggregated_stats ast
    )
    SELECT 
        CONCAT(pn.first_name_chadwick, ' ', pn.last_name_chadwick) as full_name,
        rs.p_throws as hand,
        rs.*,
        -- MATCHUP COLUMN 1: ATTACK PROFILE (Rise vs Run)
        CASE 
            WHEN vaa_plus_pct > 75 THEN 'NORTH-SOUTH (High Rise)'
            WHEN sinker_usage > 25 THEN 'EAST-WEST (Sinker/Run)'
            WHEN movement_gap_pct > 75 THEN 'DECEPTIVE (High Break)'
            ELSE 'BALANCED'
        END as attack_profile,
        -- MATCHUP COLUMN 2: ROLE IDENTITY
        CASE 
            WHEN rs.whiff_pct > 75 AND rs.location_plus_pct > 75 THEN 'DOMINANT ACE'
            WHEN rs.whiff_pct > 75 AND rs.location_plus_pct < 40 THEN 'POWER ARMS (High Risk)'
            WHEN rs.location_plus_pct > 75 AND rs.whiff_pct < 45 THEN 'PITCH TO CONTACT SURGEON'
            ELSE 'ROTATION STABILIZER'
        END as matchup_role,
        -- MATCHUP COLUMN 3: PLATOON RESISTANCE
        CASE 
            WHEN rs.neutrality_pct > 75 THEN 'MATCHUP PROOF'
            WHEN rs.neutrality_pct < 35 THEN 'PLATOON SENSITIVE'
            ELSE 'STANDARD SPLITS'
        END as platoon_identity,
        ROUND((perceived_velo_pct * 0.25 + ffour_vaa_pct * 0.25 + whiff_pct * 0.5), 0) as ffour_quality_score,
        ROUND((movement_gap_pct * 0.25 + offspeed_vaa_pct * 0.25 + whiff_pct * 0.5), 0) as offspeed_quality_score,   
        CASE 
            WHEN (ffour_vaa_pct > 80 AND ffour_usage < 20) THEN 'UNDERUSED ELITE FASTBALL'
            WHEN (bb_vaa_pct > 80 AND bb_usage < 15) THEN 'UNDERUSED ELITE BREAKING'
            WHEN (offspeed_vaa_pct > 80 AND offspeed_usage < 15) THEN 'UNDERUSED ELITE OFFSPD'
            ELSE 'OPTIMIZED'
        END as breakout_potential
    FROM ranked_stats rs
    JOIN dim_player pn ON rs.pitcher = pn.key_mlbam
    ORDER BY stuff_plus_pct DESC;
    """)
    df = pd.read_sql(query, engine)

    return run_scouting_model(df)

# Execute
pitcher_archetypes = update_dim_pitcher_archetypes(engine)



               ffour_usage  sinker_usage  ffour_vaa_pct  velo_gap_pct  \
style_cluster                                                           
0                    22.12         26.20          45.16         36.51   
1                    44.82          5.34          56.19         60.08   
2                    18.56         34.52          39.94         49.79   
3                    41.78         16.62          21.14         32.18   
4                    52.95          0.00          53.41         50.33   
5                    33.01         22.99          56.30         55.70   
6                    45.38          7.52          59.26         47.03   

               paint_pct  fb_velo  whiff_pct  suppression_pct  
style_cluster                                                  
0                  43.90    92.95      46.15            56.90  
1                  40.32    94.03      59.43            45.26  
2                  24.34    93.84      52.30            61.46  
3                  61.

In [10]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib # Recommended for saving the model later

def run_scouting_model(df):
    """
    Synthesizes pitching identity (GMM) with performance outcomes.
    Integrates tactical labels derived from cluster profiling.
    """
    
    # 1. Identity Features
    identity_features = [
        'ffour_usage', 'sinker_usage', 'bb_usage', 'offspeed_usage',
        'ffour_vaa_pct', 'sinker_vaa_pct', 'bb_vaa_pct', 'offspeed_vaa_pct',
        'velo_gap_pct', 'command_pct', 'paint_pct'
    ]
    
    # 2. Effectiveness Scores
    df['lethality_score'] = (
        (df['whiff_pct'] * 0.75) + 
        (df['suppression_pct'] * 0.20) + 
        (df['velo_pct'] * 0.05)
    ).round(1)

    # 1. Load the "Frozen" Models
    scaler = joblib.load('pitcher_scaler_v1.pkl')
    gmm = joblib.load('pitcher_model_v1.pkl')
    
    # 3. Clustering (Archetype Definition)
    #scaler = StandardScaler()
    # Filling NaNs with 0 to ensure the scaler doesn't fail
    # scaled_identity = scaler.fit_transform(df[identity_features].fillna(0))
    
    # 2. Use the frozen scaler (DO NOT use fit_transform)
    # We use .transform() so we measure new data by OLD standards
    scaled_identity = scaler.transform(df[identity_features].fillna(0))
        
    # gmm = GaussianMixture(n_components=7, random_state=42)
    # df['style_cluster'] = gmm.fit_predict(scaled_identity)
    
    # 3. Use the frozen GMM (DO NOT use fit_predict)
    # This ensures Cluster 2 ALWAYS = "Heavy Sinker Specialist"
    df['style_cluster'] = gmm.predict(scaled_identity)

    
    # # Save the scaler and model for future use
    # joblib.dump(scaler, 'pitcher_scaler_v1.pkl')
    # joblib.dump(gmm, 'pitcher_model_v1.pkl')
    # print("💾 Models saved to your project folder!")

    # 4. Define the Tactical Labels (Mapped from your Profile Results)
    cluster_map = {
            0: "Diverse Technician",
            1: "High-Octane Power Arm",
            2: "Elite Contact Manager",
            3: "Corner Specialist",
            4: "Vertical Specialist",
            5: "Versatile Tactician",
            6: "Vertical Power Lead"
        }
    df['pitcher_archetype_label'] = df['style_cluster'].map(cluster_map)

    # 5. Outlier Detection
    iso = IsolationForest(contamination=0.04, random_state=42)
    df['is_unicorn'] = iso.fit_predict(scaled_identity)
    
    def profile_pitcher_clusters(df):
        # Defining the tactical metrics we want to see
        # profile_metrics = [
        #     'style_cluster', 'fb_velo', 'ffour_usage', 'sinker_usage', 
        #     'offspeed_usage', 'whiff_pct', 'paint_pct', 'vaa_plus_pct'
        # ]
        
        profile_metrics = [
        'style_cluster', 
        # DNA
        'ffour_usage', 'sinker_usage', 'ffour_vaa_pct', 'velo_gap_pct', 'paint_pct',
        # RESULTS
        'fb_velo', 'whiff_pct', 'suppression_pct']
        
        # Calculate the mean for each cluster
        profile = df[profile_metrics].groupby('style_cluster').mean().round(2)
        
        # Sort by whiff_pct to see the "Dominance Hierarchy"
        #return profile.sort_values(by='whiff_pct', ascending=False)
        return df[profile_metrics].groupby('style_cluster').mean().round(2)
    
    # Usage:
    pitcher_profile_table = profile_pitcher_clusters(df)
    print(pitcher_profile_table)
    
    # 6. Grading Logic
    def calculate_pitcher_grade(row):
        stuff_plus = row['stuff_plus_pct']
        location_plus = row['location_plus_pct']
        
        base_score = (stuff_plus * 0.60) + (location_plus * 0.40)
        
        if row.get('is_starter') == 1: base_score += 5 
        if row.get('total_appearances', 0) < 5: base_score -= 10 

        if base_score >= 85: grade = 'A+'
        elif base_score >= 75: grade = 'A'
        elif base_score >= 60: grade = 'B'
        elif base_score >= 45: grade = 'C'
        else: grade = 'F'
            
        return grade, stuff_plus, location_plus

    df[['overall_grade', 'stuff_plus_final', 'location_plus_final']] = df.apply(
        lambda x: pd.Series(calculate_pitcher_grade(x)), axis=1
    )

    # 7. Scouting Report Generation
    def generate_scouting_report(row):
        tags = []
        # Header includes Archetype Label
        summary_header = f"[{row['overall_grade']} {row['pitcher_archetype_label']}] ({row['hand']})"
        
        # Tags
        if row['stuff_plus_pct'] >= 90: tags.append("💣 PURE FILTH")
        if row['location_plus_pct'] >= 90: tags.append("🎯 SURGEON")
        if row['is_unicorn'] == -1: tags.append("🦄 UNICORN")
        
        # Platoon Logic
        platoon = row['platoon_identity']
        if platoon == "MATCHUP PROOF": tags.append("🛡️ PLATOON NEUTRAL")
        elif platoon == "PLATOON SENSITIVE": tags.append("⚠️ SPLIT RISK")

        # Narrative Body
        analysis = f"Tactically identified as a {row['matchup_role']}. "
        if "NORTH-SOUTH" in row['attack_profile']:
            analysis += "Dominates vertically; elite matchup vs low-ball hitters. "
        elif "EAST-WEST" in row['attack_profile']:
            analysis += "East-West specialist; ideal for inducing ground balls. "

        tag_str = " | ".join(list(set(tags)))
        return tag_str, f"{summary_header} — TAGS: {tag_str} — SUMMARY: {analysis.strip()}"

    df['archetype_tags'], df['scouting_summary'] = zip(*df.apply(generate_scouting_report, axis=1))
    
    return df

def update_dim_pitcher_archetypes(engine):
    """
    SQL to extract the necessary metrics for the Python model.
    """
    query = text("""
    WITH attack_zone_stats AS (
    SELECT 
        p.*,
        -- Define Command/Paint Zones
        CASE 
            WHEN ABS(p.plate_x) <= 0.67 AND p.plate_z BETWEEN (p.sz_bot + 0.33) AND (p.sz_top - 0.33) THEN 'heart'
            WHEN ABS(p.plate_x) <= 1.1 AND p.plate_z BETWEEN (p.sz_bot - 0.33) AND (p.sz_top + 0.33) THEN 'shadow'
            WHEN ABS(p.plate_x) <= 1.5 AND p.plate_z BETWEEN (p.sz_bot - 0.75) AND (p.sz_top + 0.75) THEN 'chase'
            ELSE 'waste'
        END as attack_zone,
        CASE WHEN p.description IN ('swinging_strike', 'swinging_strike_blocked', 'missed_bunt') THEN 1 ELSE 0 END as is_whiff,
        CASE WHEN p.description IN ('swinging_strike', 'swinging_strike_blocked', 'missed_bunt', 'foul', 'foul_tip', 'hit_into_play') THEN 1 ELSE 0 END as is_swing
    FROM fact_statcast_pitches p
    ),
    vaa_base_calc AS (
        SELECT 
            az.*,
            CASE WHEN az.pitch_type IN ('FA', 'FF', 'FC') THEN 
                -ATAN((az.vz0 + (az.az * ((-az.vy0 - SQRT(az.vy0^2 - (2 * az.ay * (50 - (17/12))))) / az.ay))) / 
                (-SQRT(az.vy0^2 - (2 * az.ay * (50 - (17/12)))))) * (180/3.14159) 
            END as individual_ff_vaa
        FROM attack_zone_stats az
    ),
    aggregated_stats AS (
        SELECT 
            p.pitcher,
            p.p_throws,
            COUNT(*) as total_pitches,        
            AVG(p.release_extension) as avg_extension,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('FA', 'FF', 'FT', 'FC', 'SI') THEN p.release_speed + ((p.release_extension - 6.2) * 2) END)::numeric, 1), 0) as perceived_fb_velo,
            (AVG(CASE WHEN p.pitch_type IN ('FA', 'FF') THEN p.pfx_z * 12 END) - 
            AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'SI') THEN p.pfx_z * 12 END)) as v_break_gap_raw,
            
            ROUND(100.0 * SUM(p.is_whiff) / NULLIF(SUM(p.is_swing), 0), 2) as whiff_rate_raw,
            ROUND(100.0 * SUM(CASE WHEN p.launch_speed_angle = 6 THEN 1 ELSE 0 END) / 
                NULLIF(SUM(CASE WHEN p.type = 'X' THEN 1 ELSE 0 END), 0), 2) as barrel_rate_raw,        
            
            ROUND(100.0 * SUM(CASE WHEN p.attack_zone = 'shadow' THEN 1 ELSE 0 END) / COUNT(*), 1) as paint_raw,
            ROUND(100.0 * SUM(CASE WHEN p.attack_zone IN ('shadow', 'chase') THEN 1 ELSE 0 END) / COUNT(*), 1) as command_raw,        
            
            AVG(CASE WHEN p.stand = 'L' THEN p.estimated_woba_using_speedangle END) as xwoba_vs_lhb,
            AVG(CASE WHEN p.stand = 'R' THEN p.estimated_woba_using_speedangle END) as xwoba_vs_rhb,           
            
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('FA', 'FF', 'FT', 'FC', 'SI') THEN p.release_speed END)::numeric, 1), 0) as fb_velo,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'ST', 'SL', 'KC', 'GY', 'SV', 'CS', 'KN', 'EP') THEN p.release_speed END)::numeric, 1), 0) as offspeed_velo,                                                                                                                                                                    
            
            ROUND(100.0 * SUM(CASE WHEN p.pitch_type IN ('FA', 'FF', 'FC') THEN 1 ELSE 0 END) / COUNT(*), 1) as ffour_usage,
            ROUND(100.0 * SUM(CASE WHEN p.pitch_type IN ('SI', 'FT') THEN 1 ELSE 0 END) / COUNT(*), 1) as sinker_usage,
            ROUND(100.0 * SUM(CASE WHEN p.pitch_type IN ('CU', 'SL', 'KC', 'ST', 'SV', 'CS', 'KN') THEN 1 ELSE 0 END) / COUNT(*), 1) as bb_usage,
            ROUND(100.0 * SUM(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'ST', 'SL', 'KC', 'GY', 'SV', 'CS', 'KN', 'EP') THEN 1 ELSE 0 END) / COUNT(*), 1) as offspeed_usage,        
            
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('FA', 'FF', 'FT', 'FC', 'SI') THEN p.release_speed END)::numeric - 
                        AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'ST', 'SL', 'KC', 'GY', 'SV', 'CS', 'KN', 'EP') THEN p.release_speed END)::numeric, 1), 0) as velo_gap,        
            COALESCE(ROUND(AVG(p.individual_ff_vaa)::numeric, 2), 0) as ffour_vaa,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('SI', 'FT') THEN -ATAN((p.vz0 + (p.az * ((-p.vy0 - SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12))))) / p.ay))) / (-SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12)))))) * (180/3.14159) END)::numeric, 2), 0) as sinker_vaa,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('CU', 'SL', 'KC', 'ST', 'SV', 'CS', 'KN') THEN -ATAN((p.vz0 + (p.az * ((-p.vy0 - SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12))))) / p.ay))) / (-SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12)))))) * (180/3.14159) END)::numeric, 2), 0) as bb_vaa,
            COALESCE(ROUND(AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'EP') THEN -ATAN((p.vz0 + (p.az * ((-p.vy0 - SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12))))) / p.ay))) / (-SQRT(p.vy0^2 - (2 * p.ay * (50 - (17/12)))))) * (180/3.14159) END)::numeric, 2), 0) as offspeed_vaa,

            COUNT(DISTINCT game_pk) as total_appearances,
            (COUNT(*) / COUNT(DISTINCT game_pk)) as avg_pitches_per_app,
            CASE WHEN (COUNT(*) / COUNT(DISTINCT game_pk)) >= 40 AND COUNT(DISTINCT game_pk) >= 3 THEN 1 ELSE 0 END as is_starter,
            
            STDDEV(p.release_pos_x) as release_x_std,
            STDDEV(p.release_pos_z) as release_z_std,
            (STDDEV(p.release_pos_x) + STDDEV(p.release_pos_z)) as tunnel_raw,
            AVG(p.individual_ff_vaa - ((-0.68 * p.plate_z) - 3.8)) as vaa_above_expected_raw,       
            -- RAW STUFF+ (Process)
            ( (AVG(p.release_speed) * 0.4) + (AVG(p.release_extension) * 0.2) + (AVG(ABS(p.pfx_x)) * 12 * 0.2) + (AVG(p.pfx_z) * 12 * 0.2) ) as stuff_raw,
            -- RAW LOCATION+ (Process)
            ( (SUM(CASE WHEN p.attack_zone = 'shadow' THEN 1 ELSE 0 END)::float / COUNT(*)) * 0.6 + (SUM(CASE WHEN p.attack_zone = 'heart' THEN 0 ELSE 1 END)::float / COUNT(*)) * 0.4 ) as location_raw

        FROM vaa_base_calc p
        GROUP BY p.pitcher, p.p_throws
        HAVING COUNT(*) > 100 AND AVG(p.release_speed) > 84
    ),
    ranked_stats AS (
        SELECT 
            ast.*,
            ROUND((PERCENT_RANK() OVER (ORDER BY fb_velo))::numeric, 2) * 100 as velo_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (offspeed_usage > 0) ORDER BY offspeed_velo))::numeric, 2) * 100, 0) as offspeed_velo_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (offspeed_usage > 0) ORDER BY velo_gap))::numeric, 2) * 100, 0) as velo_gap_pct,                   
            ROUND((PERCENT_RANK() OVER (ORDER BY whiff_rate_raw))::numeric, 2) * 100 as whiff_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY barrel_rate_raw DESC))::numeric, 2) * 100 as suppression_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY command_raw))::numeric, 2) * 100 as command_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY paint_raw))::numeric, 2) * 100 as paint_pct,            
            ROUND((PERCENT_RANK() OVER (ORDER BY perceived_fb_velo))::numeric, 2) * 100 as perceived_velo_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (offspeed_usage > 0 OR sinker_usage > 0) ORDER BY v_break_gap_raw))::numeric, 2) * 100, 0) as movement_gap_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY avg_extension))::numeric, 2) * 100 as extension_pct,           
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (ffour_usage > 0) ORDER BY ffour_vaa))::numeric, 2) * 100, 0) as ffour_vaa_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (sinker_usage > 0) ORDER BY sinker_vaa DESC))::numeric, 2) * 100, 0) as sinker_vaa_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (bb_usage > 0) ORDER BY bb_vaa DESC))::numeric, 2) * 100, 0) as bb_vaa_pct,
            COALESCE(ROUND((PERCENT_RANK() OVER (PARTITION BY (offspeed_usage > 0) ORDER BY offspeed_vaa DESC))::numeric, 2) * 100, 0) as offspeed_vaa_pct,
            ROUND((100 - (ABS(COALESCE(xwoba_vs_lhb, 0.320) - COALESCE(xwoba_vs_rhb, 0.320)) * 100))::numeric, 2) as neutrality_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY tunnel_raw DESC))::numeric, 2) * 100 as tunnel_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY vaa_above_expected_raw))::numeric, 2) * 100 as vaa_plus_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY stuff_raw))::numeric, 2) * 100 as stuff_plus_pct,
            ROUND((PERCENT_RANK() OVER (ORDER BY location_raw))::numeric, 2) * 100 as location_plus_pct
        FROM aggregated_stats ast
    )
    SELECT 
        CONCAT(pn.first_name_chadwick, ' ', pn.last_name_chadwick) as full_name,
        rs.p_throws as hand,
        rs.*,
        -- MATCHUP COLUMN 1: ATTACK PROFILE (Rise vs Run)
        CASE 
            WHEN vaa_plus_pct > 75 THEN 'NORTH-SOUTH (High Rise)'
            WHEN sinker_usage > 25 THEN 'EAST-WEST (Sinker/Run)'
            WHEN movement_gap_pct > 75 THEN 'DECEPTIVE (High Break)'
            ELSE 'BALANCED'
        END as attack_profile,
        -- MATCHUP COLUMN 2: ROLE IDENTITY
        CASE 
            WHEN rs.whiff_pct > 75 AND rs.location_plus_pct > 75 THEN 'DOMINANT ACE'
            WHEN rs.whiff_pct > 75 AND rs.location_plus_pct < 40 THEN 'POWER ARMS (High Risk)'
            WHEN rs.location_plus_pct > 75 AND rs.whiff_pct < 45 THEN 'PITCH TO CONTACT SURGEON'
            ELSE 'ROTATION STABILIZER'
        END as matchup_role,
        -- MATCHUP COLUMN 3: PLATOON RESISTANCE
        CASE 
            WHEN rs.neutrality_pct > 75 THEN 'MATCHUP PROOF'
            WHEN rs.neutrality_pct < 35 THEN 'PLATOON SENSITIVE'
            ELSE 'STANDARD SPLITS'
        END as platoon_identity,
        ROUND((perceived_velo_pct * 0.25 + ffour_vaa_pct * 0.25 + whiff_pct * 0.5), 0) as ffour_quality_score,
        ROUND((movement_gap_pct * 0.25 + offspeed_vaa_pct * 0.25 + whiff_pct * 0.5), 0) as offspeed_quality_score,   
        CASE 
            WHEN (ffour_vaa_pct > 80 AND ffour_usage < 20) THEN 'UNDERUSED ELITE FASTBALL'
            WHEN (bb_vaa_pct > 80 AND bb_usage < 15) THEN 'UNDERUSED ELITE BREAKING'
            WHEN (offspeed_vaa_pct > 80 AND offspeed_usage < 15) THEN 'UNDERUSED ELITE OFFSPD'
            ELSE 'OPTIMIZED'
        END as breakout_potential
    FROM ranked_stats rs
    JOIN dim_player pn ON rs.pitcher = pn.key_mlbam
    ORDER BY stuff_plus_pct DESC;
    """)
    df = pd.read_sql(query, engine)

    return run_scouting_model(df)

# Execute
pitcher_archetypes = update_dim_pitcher_archetypes(engine)


               ffour_usage  sinker_usage  ffour_vaa_pct  velo_gap_pct  \
style_cluster                                                           
0                    22.12         26.20          45.16         36.51   
1                    44.82          5.34          56.19         60.08   
2                    18.56         34.52          39.94         49.79   
3                    41.78         16.62          21.14         32.18   
4                    52.95          0.00          53.41         50.33   
5                    33.01         22.99          56.30         55.70   
6                    45.38          7.52          59.26         47.03   

               paint_pct  fb_velo  whiff_pct  suppression_pct  
style_cluster                                                  
0                  43.90    92.95      46.15            56.90  
1                  40.32    94.03      59.43            45.26  
2                  24.34    93.84      52.30            61.46  
3                  61.

Hitters model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sqlalchemy.engine import Engine
from sqlalchemy import text
from datetime import date, timedelta
from datetime import datetime
import joblib
import os

from dotenv import load_dotenv

def create_dim_hitter_archetypes(engine: Engine):
    try:
        print("💾 Creating dim_hitter_archetype...")
        def run_hitter_scouting_model(df):
            """
            Stabilized Hitter Model: Uses Bayesian-weighted metrics to define 
            archetypes and calculate grades.
            """
            
            # 1. Identity Features (The DNA)
            # We use 'stabilized_ev' and 'neutrality_raw' to ensure cluster stability
            identity_features = [
                'chase_pct_raw', 'zone_swing_raw', 'first_pitch_swing_raw',
                'avg_la', 'pull_pct_raw', 'two_strike_contact_raw',
                'woba_vs_hard', 'woba_vs_break', 'woba_vs_offspeed',
                'neutrality_raw', 'stabilized_ev'
            ]
            
            # 2. Effectiveness Scores (Grade Components)
            # Power is now grounded in stabilized EV
            df['power_score'] = (
                (df['ev_pct'] * 0.50) + 
                (df['barrel_pct'] * 0.50)
            ).round(1)
            
            # Eye combines discipline with 'Battle' (2-strike) ability
            df['discipline_score'] = (
                (df['discipline_pct'] * 0.60) + 
                (df['battle_pct'] * 0.40)
            ).round(1)
            
            # 1. Load the .env file 
            # Since .env is in the same folder as this script, we can just load it
            load_dotenv()

            # 2. Get the path from the environment variable
            # If it's not found, we provide a 'fallback' default
            input_folder = os.getenv('INPUT_DIR', '../input')

            # 3. Construct the full paths to your files
            # We use abspath to make sure Python doesn't get confused by the current working directory
            base_path = os.path.dirname(os.path.abspath(__file__))
            scaler_path = os.path.abspath(os.path.join(base_path, input_folder, 'hitter_scaler_v1.pkl'))
            model_path = os.path.abspath(os.path.join(base_path, input_folder, 'hitter_model_v1.pkl'))

            # 4. Load the files
            scaler = joblib.load(scaler_path)
            gmm    = joblib.load(model_path)

            # 3. Clustering (Archetype Definition)
            scaler = StandardScaler()
            # Filling NaNs with league average proxies for safety
            #scaled_data = scaler.fit_transform(df[identity_features].fillna(df[identity_features].median()))
            scaled_data = scaler.transform(df[identity_features].fillna(df[identity_features].median()))
            
            # 7 Clusters allows for enough nuance (Sluggers, Pests, Specialists, etc.)
            gmm = GaussianMixture(n_components=7, random_state=42, n_init=10)
            df['hitter_cluster'] = gmm.predict(scaled_data)

            # # SAVE THE MODELS (Run once, then comment out)
            # joblib.dump(scaler, 'hitter_scaler_v1.pkl')
            # joblib.dump(gmm, 'hitter_model_v1.pkl')
                        
            # 4. Professional Hitter Archetype Mapping
            # Note: Validate these against your printed profile table!
            hitter_map = {
                0: "Discipline Specialist",  # Low chase, high walks, moderate power
                1: "Pull-Side Power Slugger", # High LA, High Pull%, High EV
                2: "Contact-Oriented Pest",   # High 2-strike contact, low LA, low K%
                3: "All-Fields Technician",   # Neutrality high, line drive plane
                4: "Aggressive Free-Swinger", # High first-pitch swing, high chase
                5: "Modern Three-True-Outcome", # High LA, High Whiff, High EV
                6: "High-Traffic Slap Hitter" # Low LA, high zone swing, high speed/contact
            }
            df['hitter_archetype_label'] = df['hitter_cluster'].map(hitter_map)
            
            # 5. Outlier Detection (Unicorns)
            iso = IsolationForest(contamination=0.03, random_state=42)
            df['is_hitter_unicorn'] = iso.fit_predict(scaled_data)
            
            # def profile_hitter_clusters(df):
            #     profile_metrics = [
            #         'hitter_cluster',
            #         # DNA (Identity)
            #         'chase_pct_raw', 'zone_swing_raw', 'avg_la', 'pull_pct_raw', 'two_strike_contact_raw',
            #         # Results
            #         'ev_pct', 'barrel_pct', 'woba_reliability_pct', 'discipline_pct'
            #     ]
            #     return df[profile_metrics].groupby('hitter_cluster').mean().round(2)

            # # Usage: Run this and print the result to see the groups
            # hitter_profile_table = profile_hitter_clusters(df)
            # print(hitter_profile_table)
            
            # 5. Overall Grade Calculation
            def calculate_scouting_grade(row, mode='hitter'):
                """
                Applies the 20-80 Scouting Scale (Standardized).
                A+ = 80 Grade (Generational)
                A  = 70 Grade (Elite All-Star)
                B  = 60 Grade (Plus)
                C  = 50 Grade (Average)
                """
                # 1. Get the combined Z-score from SQL
                z_score = row.get('combined_scouting_z', 0)
                
                # 2. Determine sample size based on player type
                if mode == 'hitter':
                    sample = row.get('total_pitches_faced', 0)
                else:
                    sample = row.get('total_pitches_thrown', 0)
                    
                # 3. Apply Reliability Penalty (Prevents small-sample "fluke" A's)
                if sample < 500:
                    z_score -= 1.0  # Moves a lucky 'A' down to a 'B' or 'C'
                elif sample < 1000:
                    z_score -= 0.4  # Slight penalty for unproven consistency
                    
                # 4. Universal Grade Thresholds
                if z_score >= 2.4:
                    return 'A+'
                elif z_score >= 1.5:
                    return 'A'
                elif z_score >= 0.6:
                    return 'B'
                elif z_score >= -0.5:
                    return 'C'
                else:
                    return 'D/F'

            # Usage:
            df['overall_grade'] = df.apply(lambda x: calculate_scouting_grade(x, mode='hitter'), axis=1)


            # # 6. Scouting Report Generation
            # def generate_hitter_report(row):
            #     tags = []
                
            #     # Logic for tags
            #     if row['power_score'] >= 90: tags.append("🔥 ELITE POWER")
            #     if row['discipline_score'] >= 90: tags.append("🎯 DISCIPLINE MASTER")
            #     if row['two_strike_identity'] == 'ELITE SPOILER': tags.append("🦟 PEST")
            #     if row['neutrality_pct'] > 85: tags.append("🛡️ MATCHUP PROOF")
            #     if row['is_hitter_unicorn'] == -1: tags.append("🦄 UNICORN")

            #     # Vertical Profile Analysis
            #     v_desc = f"Attacks the {row['vertical_profile']}."
                
            #     # Build Summary
            #     conf_prefix = "PROVISIONAL: " if "PROVISIONAL" in row['data_confidence'] else ""
            #     header = f"[{row['overall_grade']}] {conf_prefix}{row['full_name']} ({row['hand']})"
                
            #     tag_str = " | ".join(tags)
            #     body = f"{v_desc} Handles {row['swing_plane']} path. "
                
            #     # Platoon Insight
            #     if row['neutrality_pct'] < 30:
            #         body += f"Extreme platoon splits detected; high risk against {row['hand']}HP. "
            #     else:
            #         body += "Balanced splits make them difficult to platoon against. "

            #     return tag_str, f"{header}\nTAGS: {tag_str}\nSUMMARY: {body}"

            # df['hitter_tags'], df['hitter_summary'] = zip(*df.apply(generate_hitter_report, axis=1))
            
            # return df
            
            # 6. Scouting Report Generation (Updated for new labels)
            def generate_hitter_report(row):
                tags = []
                header = f"[{row['overall_grade']} {row['hitter_archetype_label']}] ({row['hand']})"
                
                if row['power_score'] >= 90: tags.append("🔥 ELITE POWER")
                if row['discipline_score'] >= 90: tags.append("🎯 DISCIPLINE MASTER")
                if row['two_strike_identity'] == 'ELITE SPOILER': tags.append("🦟 PEST")
                if row['is_hitter_unicorn'] == -1: tags.append("🦄 UNICORN")

                tag_str = " | ".join(tags)
                body = f"Tactically identified as a {row['hitter_archetype_label']}. "
                body += f"Attacks the {row['vertical_profile']} with a {row['swing_plane']} path. "
                
                return tag_str, f"{header}\nTAGS: {tag_str}\nSUMMARY: {body}"

                df['hitter_tags'], df['hitter_summary'] = zip(*df.apply(generate_hitter_report, axis=1))
                
                return df

        def update_dim_hitter_archetypes(engine):
            """
            SQL to extract the necessary metrics for the Python model.
            """
            query = text("""
            WITH constants AS (
                SELECT 
                    0.312 as lg_woba,    -- Actual 2024-25 league average
                    88.4  as lg_ev,      -- Current league avg exit velocity
                    0.045 as woba_sd,    -- Fixed Standard Deviation for wOBA
                    3.8   as ev_sd,      -- Fixed Standard Deviation for Exit Velocity
                    0.040 as barrel_sd,  -- Fixed Standard Deviation for Barrel Rate
                    500   as m_woba,     -- Reliability threshold for wOBA
                    150   as m_ev        -- Reliability threshold for EV
            ),
            attack_zone_stats AS (
                SELECT 
                    p.*,
                    CASE WHEN p.strikes = 2 THEN 1 ELSE 0 END as is_two_strike,
                    CASE WHEN p.inning >= 7 AND ABS(p.bat_score - p.fld_score) <= 2 THEN 1 ELSE 0 END as is_clutch,
                    CASE WHEN p.zone > 9 AND p.description IN ('swinging_strike', 'foul', 'hit_into_play', 'swinging_strike_blocked') THEN 1 ELSE 0 END as is_chase,
                    CASE WHEN p.zone <= 9 AND p.description IN ('swinging_strike', 'foul', 'hit_into_play', 'swinging_strike_blocked') THEN 1 ELSE 0 END as is_zone_swing,
                    CASE 
                        WHEN (p.stand = 'R' AND p.hc_x < 125) OR (p.stand = 'L' AND p.hc_x > 125) THEN 1 
                        ELSE 0 
                    END as is_pull
                FROM fact_statcast_pitches p
            ),
            hitter_base_stats AS (
                SELECT 
                    p.batter,
                    p.stand AS bat_side,
                    COUNT(*) AS total_pitches_faced,
                    COUNT(CASE WHEN p.type = 'X' THEN 1 END) as balls_in_play,
                    -- Bayesian Stabilized Metrics
                    ((SUM(p.woba_value) + (MAX(c.m_woba) * MAX(c.lg_woba))) / (COUNT(*) + MAX(c.m_woba))) as stabilized_woba,
                    ((SUM(p.launch_speed) + (MAX(c.m_ev) * MAX(c.lg_ev))) / (NULLIF(COUNT(CASE WHEN p.type = 'X' THEN 1 END), 0) + MAX(c.m_ev))) as stabilized_ev,
                    -- Raw Stats for Clustering & Context
                    AVG(CASE WHEN p.pitch_type IN ('FF', 'FA', 'FT', 'SI', 'FC') THEN p.woba_value END) AS woba_vs_hard,
                    AVG(CASE WHEN p.pitch_type IN ('SL', 'ST', 'CU', 'KC', 'SV', 'CS', 'GY', 'KN') THEN p.woba_value END) AS woba_vs_break,
                    AVG(CASE WHEN p.pitch_type IN ('CH', 'FS', 'FO', 'SC', 'EP') THEN p.woba_value END) AS woba_vs_offspeed,            
                    ROUND(SUM(p.is_chase)::numeric / NULLIF(SUM(CASE WHEN p.zone > 9 THEN 1 ELSE 0 END), 0) * 100, 1) as chase_pct_raw,
                    ROUND(SUM(p.is_zone_swing)::numeric / NULLIF(SUM(CASE WHEN p.zone <= 9 THEN 1 ELSE 0 END), 0) * 100, 1) as zone_swing_raw,
                    ROUND(SUM(CASE WHEN p.balls = 0 AND p.strikes = 0 AND p.description IN ('swinging_strike', 'foul', 'hit_into_play') THEN 1 ELSE 0 END)::numeric / 
                        NULLIF(SUM(CASE WHEN p.balls = 0 AND p.strikes = 0 THEN 1 ELSE 0 END), 0) * 100, 1) as first_pitch_swing_raw,
                    ROUND(SUM(CASE WHEN p.description IN ('swinging_strike', 'swinging_strike_blocked') THEN 1 ELSE 0 END)::numeric / 
                        NULLIF(SUM(CASE WHEN p.description IN ('swinging_strike', 'foul', 'hit_into_play') THEN 1 ELSE 0 END), 0) * 100, 1) as whiff_pct_hitter,      
                    AVG(p.launch_speed) AS avg_ev,
                    AVG(p.launch_angle) AS avg_la,
                    SUM(CASE WHEN p.launch_speed_angle = 6 THEN 1 ELSE 0 END)::float / NULLIF(SUM(CASE WHEN p.type = 'X' THEN 1 ELSE 0 END), 0) AS barrel_rate_raw,
                    AVG(CASE WHEN p.plate_z > (p.sz_top + p.sz_bot)/2 THEN p.woba_value END) as woba_high_raw,
                    AVG(CASE WHEN p.plate_z <= (p.sz_top + p.sz_bot)/2 THEN p.woba_value END) as woba_low_raw,      
                    ROUND(SUM(CASE WHEN p.is_two_strike = 1 AND p.description IN ('foul', 'hit_into_play') THEN 1 ELSE 0 END)::numeric / 
                        NULLIF(SUM(CASE WHEN p.is_two_strike = 1 AND p.description IN ('swinging_strike', 'foul', 'hit_into_play') THEN 1 ELSE 0 END), 0) * 100, 1) as two_strike_contact_raw,       
                    AVG(CASE WHEN p.is_clutch = 1 THEN p.woba_value END) as clutch_woba_raw,
                    100 - (ABS(COALESCE(AVG(CASE WHEN p.p_throws = 'L' THEN p.woba_value END), 0.320) - 
                            COALESCE(AVG(CASE WHEN p.p_throws = 'R' THEN p.woba_value END), 0.320)) * 100) as neutrality_raw,
                    ROUND(SUM(p.is_pull)::numeric / NULLIF(SUM(CASE WHEN p.type = 'X' THEN 1 ELSE 0 END), 0) * 100, 1) as pull_pct_raw
                FROM attack_zone_stats p, constants c
                GROUP BY p.batter, p.stand
                HAVING COUNT(*) > 150
            ),
            hitter_ranked AS (
                SELECT 
                    hb.*,
                    -- UNIVERSAL Z-SCORE CALCULATION
                    (hb.stabilized_ev - 88.5) / 3.5 as ev_z,
                    (hb.stabilized_woba - 0.320) / 0.045 as woba_z,
                    (hb.barrel_rate_raw - 0.075) / 0.040 as barrel_z,       
                    -- Percentiles (kept for existing columns)
                    ROUND((PERCENT_RANK() OVER (ORDER BY avg_ev))::numeric, 2) * 100 AS ev_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY barrel_rate_raw))::numeric, 2) * 100 AS barrel_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY stabilized_woba))::numeric, 2) * 100 AS woba_reliability_pct,
                    ROUND((1 - PERCENT_RANK() OVER (ORDER BY chase_pct_raw))::numeric, 2) * 100 AS discipline_pct,     
                    ROUND((PERCENT_RANK() OVER (ORDER BY first_pitch_swing_raw))::numeric, 2) * 100 AS aggression_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY woba_high_raw))::numeric, 2) * 100 AS high_ball_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY woba_low_raw))::numeric, 2) * 100 AS low_ball_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY pull_pct_raw))::numeric, 2) * 100 AS pull_pct,      
                    ROUND((PERCENT_RANK() OVER (ORDER BY two_strike_contact_raw))::numeric, 2) * 100 AS battle_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY clutch_woba_raw))::numeric, 2) * 100 AS clutch_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY neutrality_raw))::numeric, 2) * 100 as neutrality_pct,       
                    ROUND((PERCENT_RANK() OVER (ORDER BY woba_vs_hard))::numeric, 2) * 100 as woba_hard_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY woba_vs_break))::numeric, 2) * 100 as woba_break_pct,
                    ROUND((PERCENT_RANK() OVER (ORDER BY woba_vs_offspeed))::numeric, 2) * 100 as woba_offspeed_pct
                FROM hitter_base_stats hb
            )
            SELECT 
                CONCAT(pn.first_name_chadwick, ' ', pn.last_name_chadwick) AS full_name,
                hr.bat_side as hand,
                hr.*,
                -- UNIVERSAL ANCHOR: Matches Pitcher Model Scale
                ((hr.woba_z * 0.5) + (hr.barrel_z * 0.3) + (hr.ev_z * 0.2)) as combined_scouting_z,
                CASE 
                    WHEN total_pitches_faced > 1500 THEN 'VERIFIED ELITE SAMPLE'
                    WHEN total_pitches_faced > 600 THEN 'STABILIZED'
                    ELSE 'PROVISIONAL (Small Sample)'
                END as data_confidence,
                CASE 
                    WHEN avg_la > 18 THEN 'UPPERCUT (Flyball)'
                    WHEN avg_la < 8 THEN 'DOWNWARD (Groundball)'
                    ELSE 'LEVEL (Line Drive)'
                END AS swing_plane,
                CASE 
                    WHEN battle_pct > 80 THEN 'ELITE SPOILER'
                    WHEN battle_pct < 25 THEN 'FREE SWINGER'
                    ELSE 'STANDARD'
                END AS two_strike_identity,
                CASE
                    WHEN high_ball_pct > 75 AND low_ball_pct < 40 THEN 'HIGH-BALL HUNTER'
                    WHEN low_ball_pct > 75 AND high_ball_pct < 40 THEN 'LOW-BALL GOLFER'
                    ELSE 'ALL-ZONE THREAT'
                END AS vertical_profile
            FROM hitter_ranked hr
            JOIN dim_player pn ON hr.batter = pn.key_mlbam
            ORDER BY combined_scouting_z DESC;
            """)
            df = pd.read_sql(query, engine)
            
            return run_hitter_scouting_model(df)

        # Execute function
        hitter_archetypes = update_dim_hitter_archetypes(engine)

        # Add the calculation_date
        hitter_archetypes['calculation_date'] = datetime.now()

        # Load to SQL
        hitter_archetypes.to_sql(
        'dim_hitter_archetypes', 
        engine, 
        if_exists='replace',
        index=False, 
        chunksize=5000
        )
        
        print(f"   ✅ Successfully added {len(hitter_archetypes)} new rows of data.")

    except Exception as e:
        print(f"   ❌ ETL Failed during extraction or loading: {e}")
        
#create_dim_hitter_archetypes(engine)

💾 Creating dim_hitter_archetype...
   ❌ ETL Failed during extraction or loading: name '__file__' is not defined


In [1]:
import streamlit as st
from sqlalchemy.engine import Engine
import sys
from pathlib import Path
import os

# Add source directory to path
app_dir = Path(__file__).parent.parent
source_dir = app_dir.parent
sys.path.insert(0, str(source_dir))

from connection_engine import create_connection_postgresql
from dotenv import load_dotenv

# Load environment variables - check multiple locations
env_paths = [
    source_dir / "utils" / ".env",  # Original location: Source/utils/.env
    app_dir / "utils" / ".env",      # App location: Source/app/utils/.env
    source_dir / ".env"              # Fallback: Source/.env
]

env_loaded = False
for env_path in env_paths:
    if env_path.exists():
        load_dotenv(dotenv_path=str(env_path), override=True)
        env_loaded = True
        break

# If no .env file found, try default load_dotenv (current directory)
if not env_loaded:
    load_dotenv(override=True)
    

def get_db_engine() -> Engine:
    """Get or create database engine"""
    if st.session_state.db_engine is None:
        try:
            # Ensure .env is loaded before creating connection (check multiple locations)
            env_paths = [
                source_dir / "utils" / ".env",  # Source/utils/.env
                app_dir / "utils" / ".env",      # Source/app/utils/.env
                source_dir / ".env"              # Source/.env
            ]
            
            env_loaded = False
            for env_path in env_paths:
                if env_path.exists():
                    load_dotenv(dotenv_path=str(env_path), override=True)
                    env_loaded = True
                    break
            
            if not env_loaded:
                load_dotenv(override=True)  # Try default location
            
            st.session_state.db_engine = create_connection_postgresql()
        except KeyError as e:
            # Provide helpful error message with possible locations
            possible_locations = [
                str(source_dir / "utils" / ".env"),
                str(app_dir / "utils" / ".env"),
                str(source_dir / ".env")
            ]
            st.error(
                f"Database connection error: Missing environment variable '{e}'. "
                f"Please check your .env file. Expected locations:\n"
                f"- {possible_locations[0]}\n"
                f"- {possible_locations[1]}\n"
                f"- {possible_locations[2]}"
            )
            return None
        except Exception as e:
            st.error(f"Database connection error: {e}")
            return None
    return st.session_state.db_engine




NameError: name '__file__' is not defined

In [2]:
import os
import sys
from pathlib import Path
from sqlalchemy.engine import Engine
from dotenv import load_dotenv

# 1. HANDLE DIRECTORY PATHS (Notebook compatible)
try:
    # If running as a .py script
    base_path = Path(__file__).resolve().parent
except NameError:
    # If running in a Jupyter Notebook (.ipynb)
    base_path = Path(os.getcwd()).resolve()

# Adjust these based on your actual folder structure
# Based on your previous code: Source/app/pages/...
app_dir = base_path 
source_dir = app_dir.parent
sys.path.insert(0, str(source_dir))

# 2. IMPORT DATABASE UTILS
# Note: Ensure connection_engine.py is in your source_dir
try:
    from connection_engine import create_connection_postgresql
    print("✅ Successfully imported connection_engine")
except ImportError as e:
    print(f"❌ Import Error: {e}")
    print(f"Current sys.path: {sys.path[0]}")

# 3. LOAD ENVIRONMENT VARIABLES
env_paths = [
    source_dir / "utils" / ".env",
    app_dir / "utils" / ".env",
    source_dir / ".env",
    base_path / ".env"
]

env_loaded = False
for env_path in env_paths:
    if env_path.exists():
        load_dotenv(dotenv_path=str(env_path), override=True)
        print(f"✅ Loaded .env from: {env_path}")
        env_loaded = True
        break

if not env_loaded:
    print("⚠️ No .env file found in search paths. Falling back to default.")
    load_dotenv(override=True)

# 4. DATABASE ENGINE FUNCTION (Modified for Notebook testing)
# We use a global variable instead of st.session_state
_DB_ENGINE = None

def get_db_engine() -> Engine:
    """Get or create database engine (Notebook version)"""
    global _DB_ENGINE
    
    if _DB_ENGINE is None:
        try:
            print("Connecting to database...")
            _DB_ENGINE = create_connection_postgresql()
            print("✅ Database connection successful!")
        except Exception as e:
            print(f"❌ Database connection error: {e}")
            return None
    return _DB_ENGINE

# 5. TEST THE CONNECTION
if __name__ == "__main__":
    engine = get_db_engine()
    if engine:
        try:
            with engine.connect() as conn:
                print("🔗 Connection test: Valid")
        except Exception as e:
            print(f"❌ Connection test failed: {e}")

✅ Successfully imported connection_engine
✅ Loaded .env from: C:\Development\python_projects\Code\projects\baseball_analytics\Sandbox\.env
Connecting to database...
🛜  Connecting to the database...
   ✅ Database connection established.
✅ Database connection successful!
🔗 Connection test: Valid
